# RoBERTa BCE

## Variables d'environnement

In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# Id des GPU disponibles : 0 et 1

## Importation

In [18]:
import os
import time
import datetime
from typing import Any, Union, Dict, List
import uuid
import json

import pandas as pd
import numpy as np
import torch
from torch import nn
import torchtext
import nltk
import sklearn
import transformers
import torchmetrics as tm
from torchmetrics import MetricCollection, Metric, Accuracy, Precision, Recall, AUROC, HammingDistance, F1, ROC, AUC, PrecisionRecallCurve


from loguru import logger
from tqdm.auto import tqdm
tqdm.pandas()

# import warnings
# warnings.filterwarnings("ignore")

## Constantes

In [25]:
CUSTOME_NAME = "roberta-bce"

# Dataset
DATA_DIR_PATH = os.path.abspath("../../data")
TRAIN_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2019-train.csv")
TEST_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2019-test.csv")
LABEL_LIST = ['toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
            'identity_attack', 'insult', 'threat']
IDENTITY_LIST = ['male', 'female', 'transgender', 'other_gender', 'heterosexual',
                'homosexual_gay_or_lesbian', 'bisexual','other_sexual_orientation',
                'christian', 'jewish', 'muslim', 'hindu','buddhist', 'atheist',
                'other_religion', 'black', 'white', 'asian', 'latino',
                'other_race_or_ethnicity', 'physical_disability',
                'intellectual_or_learning_disability',
                'psychiatric_or_mental_illness','other_disability']
SELECTED_IDENTITY_LIST = ['male', 'female', 'black', 'white', 'homosexual_gay_or_lesbian',
                    'christian', 'jewish', 'muslim', 'psychiatric_or_mental_illness']

# Session
SESSION_DIR_PATH = os.path.abspath("../../session")
SESSION_DATETIME = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S-%f")
SESSION_NAME = f"{CUSTOME_NAME}_{SESSION_DATETIME}"
CURRENT_SESSION_DIR_PATH = os.path.join(SESSION_DIR_PATH, SESSION_NAME)
# Créer le dossier de la session
os.makedirs(CURRENT_SESSION_DIR_PATH, exist_ok=True)

# Architecture de fichier dans `CURRENT_SESSION_DIR_PATH`
LOG_FILE_NAME = f"{SESSION_NAME}.loguru.log"
MODEL_FILE_NAME = f"{SESSION_NAME}.model"
TEST_FILE_NAME = f"{SESSION_NAME}.test.csv"
METRIC_FILE_NAME = f"{SESSION_NAME}.metric.json"
LOG_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, LOG_FILE_NAME)
MODEL_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, MODEL_FILE_NAME)
TEST_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, TEST_FILE_NAME)
METRIC_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, METRIC_FILE_NAME)

# CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Logging

In [23]:
logger.add(LOG_FILE_PATH, level="TRACE")
logger.info(f"{SESSION_NAME=}")
logger.info(f"{TRAIN_DATASET_PATH=}")
logger.info(f"{TEST_DATASET_PATH=}")
logger.info(f"{CURRENT_SESSION_DIR_PATH=}")

2022-03-24 18:36:51.105 | INFO     | __main__:<cell line: 2>:2 - SESSION_NAME='roberta-bce_2022-03-24T18-36-33-957856'
2022-03-24 18:36:51.108 | INFO     | __main__:<cell line: 3>:3 - TRAIN_DATASET_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/data/jigsaw2019-train.csv'
2022-03-24 18:36:51.110 | INFO     | __main__:<cell line: 4>:4 - TEST_DATASET_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/data/jigsaw2019-test.csv'
2022-03-24 18:36:51.112 | INFO     | __main__:<cell line: 5>:5 - CURRENT_SESSION_DIR_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/session/roberta-bce_2022-03-24T18-36-33-957856'


## Vérifier la cohérence de l'architecture et l'accès aux ressources

In [21]:
logger.info(f"Checking consistency...")

# Vérifier l'accès aux datasets
if not os.path.exists(TRAIN_DATASET_PATH):
    logger.critical(f"Train dataset does not exist !")
    raise RuntimeError("Train dataset does not exist !")
if not os.path.exists(TEST_DATASET_PATH):
    logger.critical(f"Test dataset does not exist !")
    raise RuntimeError("Test dataset does not exist !")
logger.success("Datasets are reachable")

# Vérifier l'accès aux GPU
GPU_IS_AVAILABLE = torch.cuda.is_available()
GPU_COUNT = torch.cuda.device_count()
logger.info(f"{GPU_IS_AVAILABLE=}")
logger.info(f"{GPU_COUNT=}")
if not GPU_IS_AVAILABLE:
    logger.critical("GPU and CUDA are not available !")
    raise RuntimeError("GPU and CUDA are not available !")
logger.success("GPU and CUDA are available")
logger.info(f"{device=}")
for gpu_id in range(GPU_COUNT):
    gpu_name = torch.cuda.get_device_name(0)
    logger.info(f"GPU {gpu_id} : {gpu_name}")

2022-03-24 18:34:14.978 | INFO     | __main__:<cell line: 1>:1 - Checking consistency...
2022-03-24 18:34:14.983 | SUCCESS  | __main__:<cell line: 10>:10 - Datasets are reachable
2022-03-24 18:34:14.986 | INFO     | __main__:<cell line: 15>:15 - GPU_IS_AVAILABLE=True
2022-03-24 18:34:14.988 | INFO     | __main__:<cell line: 16>:16 - GPU_COUNT=1
2022-03-24 18:34:14.990 | SUCCESS  | __main__:<cell line: 20>:20 - GPU and CUDA are available
2022-03-24 18:34:14.992 | INFO     | __main__:<cell line: 21>:21 - device=device(type='cuda')
2022-03-24 18:34:14.994 | INFO     | __main__:<cell line: 22>:24 - GPU 0 : NVIDIA TITAN X (Pascal)


## Dataset

In [47]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, index_col=0)

In [48]:
# Remplacer toutes les colonnes correspondantes aux labels par 1 ou 0
# si la probabilité est supérieure ou égale à 0.5 ou non
train_df[LABEL_LIST] = (train_df[LABEL_LIST]>=0.5).astype(int)

In [49]:
TOKENIZER_NAME = "roberta-base"
logger.info(f"{TOKENIZER_NAME=}")
tokenizer = transformers.AutoTokenizer.from_pretrained(TOKENIZER_NAME)

def text_to_token_and_mask(input_text):
    tokenization_dict = tokenizer.encode_plus(input_text,
                                            add_special_tokens=True,
                                            max_length=128,
                                            padding='max_length',
                                            truncation=True,
                                            return_attention_mask=True,
                                            return_tensors='pt')
    token_list = tokenization_dict["input_ids"].flatten()
    attention_mask = tokenization_dict["attention_mask"].flatten()
    return (token_list, attention_mask)

2022-03-24 19:46:13.452 | INFO     | __main__:<cell line: 2>:2 - TOKENIZER_NAME='roberta-base'


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [50]:
class JigsawTrainDataset(torch.utils.data.Dataset):
    def __init__(self, data_df):
        self.data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        comment = self.data.iloc[index]["comment_text"]
        label = torch.tensor(self.data.iloc[index][LABEL_LIST].tolist(), dtype=torch.float)
        
        token_list, attention_mask = text_to_token_and_mask(comment)

        return dict(ids=token_list, mask=attention_mask, labels=label)

## Model

In [ ]:
def set_lr(optim, lr):
    '''
    Set the learning rate in the optimizer
    '''
    for g in optim.param_groups:
        g['lr'] = lr
    return optim

In [ ]:
# Transformer class and functions for models and predictions

class TransformerClassifierStack(nn.Module):
    def __init__(self, tr_model, nb_labels, dropout_prob=0.4, freeze=False):
        super().__init__()
        self.tr_model = tr_model

        # Stack features of 4 last encoders
        self.hidden_dim = tr_model.config.hidden_size * 4

        # hidden linear for the classification
        self.dropout = nn.Dropout(dropout_prob)
        self.hl = nn.Linear(self.hidden_dim, self.hidden_dim)

        # Last Linear for the classification
        self.last_l = nn.Linear(self.hidden_dim, nb_labels)

        # freeze all the parameters if necessary
        for param in self.tr_model.parameters():
            param.requires_grad = not freeze

        # init learning params of last layers
        torch.nn.init.xavier_uniform_(self.hl.weight)
        torch.nn.init.xavier_uniform_(self.last_l.weight)

    def forward(self, ids, mask):
        # ids = [batch_size, padded_seq_len]
        # mask = [batch_size, padded_seq_len]
        # mask: avoid to make self attention on padded data
        tr_output = self.tr_model(input_ids=ids,
                                  attention_mask=mask,
                                  output_hidden_states=True)

        # Get all the hidden states
        hidden_states = tr_output['hidden_states']

        # hs_* = [batch_size, padded_seq_len, 768]
        hs_1 = hidden_states[-1][:, 0, :]
        hs_2 = hidden_states[-2][:, 0, :]
        hs_3 = hidden_states[-3][:, 0, :]
        hs_4 = hidden_states[-4][:, 0, :]

        # features_vec = [batch_size, 768 * 4]
        features_vec = torch.cat([hs_1, hs_2, hs_3, hs_4], dim=-1)

        x = self.dropout(features_vec)
        x = self.hl(x)

        # x = [batch_size, 768 * 4]
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.last_l(x)

        # x = [batch_size, 1]
        return x

def load_roberta_model(nb_labels):
    '''
    Load RoBERTa model without any checkpoint
    RoBERTa for finetuning
    '''

    tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
    tr_model = transformers.AutoModel.from_pretrained('roberta-base')
    model = TransformerClassifierStack(tr_model, nb_labels)
    return model, tokenizer


def load_roberta_pretrained(path, nb_labels, lr=2e-5):
    '''
    Load RoBERTa from checkout point (already trained on Hate Speech tasks)
    '''
    tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
    tr_model = transformers.AutoModel.from_pretrained('roberta-base')
    model = TransformerClassifierStack(tr_model, nb_labels)

    loaded = torch.load(path)
    model.load_state_dict(loaded['state_dict'])

    optimizer = transformers.AdamW(model.parameters(), lr=lr)
    optimizer.load_state_dict(loaded['optimizer_dict'])
    optimizer = set_lr(optimizer, lr)

    return model, tokenizer, optimizer

def preds_fn(batch, model, device):
    '''
    Get the predictions for one batch according to the model
    '''
    b_input = batch['ids'].to(device)
    b_mask = batch['mask'].to(device)

    return model(b_input, b_mask)

In [ ]:
# Load the model
model, tokenizer = load_roberta_model(nb_labels=len(LABEL_LIST))